Function to download trip id:s from Ride with GPS data API


In [4]:
import requests
import time

def fetch_all_trip_ids(response):
    trip_ids = set()
    for route in response['results']:
        if 'trip' in route:
            trip_ids.add(route['trip']['id'])
    return trip_ids

#define start location for the query and distance (20 km)
def fetch_trip_ids(start_location="67.62033,24.26553", start_distance=20):
    url = "https://ridewithgps.com/find/search.json"
    offset = 0
    #add request limit of 300
    request_limit = 300
    trip_ids = set()

    stopper = 0
    stopper2 = 0
    while True:
        #add to stopper to change start location after two and four iterations
        stopper2 += 1
        data = {
            "search": {
                "keywords": "",
                "start_location": start_location,
                "start_distance": str(start_distance),
                "elevation_max": "",
                "elevation_min": "",
                "length_max": "",
                "length_min": "",
                "offset": str(offset),
                "limit": str(request_limit),
                "sort_by": "length asc"
            },
            "apikey": "********",
            "version": "2",
            "auth_token": "*****************" #replace with toke received from Ride with GPS
        }

        response = requests.post(url, json=data)
        if response.status_code == 200:
            data = response.json()
            new_trip_ids = fetch_all_trip_ids(data)
            trip_ids.update(new_trip_ids)
            
            offset += request_limit
            time.sleep(5)
            
        
        elif response.status_code == 400:
            print(f"Received 400 status code. Retrying after 5 seconds...")
            time.sleep(5)
            stopper += 1
            if stopper == 3:
                break
            else:
                continue
        else:
            print(f"Error fetching trip IDs. Status code: {response.status_code}")
            break
    #change query locations after two and four iterations
        start_location = "67.91606,24.21740"
    if stopper2 == 4:
        start_location ="68.14865,23.94110"
    return trip_ids

# Example usage
all_trip_ids = fetch_trip_ids()
print(len(all_trip_ids))

Received 400 status code. Retrying after 5 seconds...
Received 400 status code. Retrying after 5 seconds...
Received 400 status code. Retrying after 5 seconds...
Total unique trip IDs: 975


In [81]:
import requests
import geopandas as gpd
import time
from shapely.geometry import Point, LineString, Polygon

Function to download trip details

In [82]:
import pandas as pd
import geopandas as gpd

#function to fetch trip id:s
def fetch_trip_data(trip_id):
    base_url = "https://ridewithgps.com/trips/{}.json"
    url = base_url.format(trip_id)
    response = requests.get(url)
    if response.status_code == 200:
        trip_data = response.json()
        return trip_data
    else:
        print(f"Error fetching trip data for trip ID {trip_id}: {response.status_code}")
        return None

# sample list of trip IDs
trip_ids = all_trip_ids
# fetch trip data for each trip ID
trip_data_list = []
for trip_id in trip_ids:
    trip_data = fetch_trip_data(trip_id)
    if trip_data:
        trip_data_list.append(trip_data)
    else:
        print(f"Failed to fetch data for trip ID {trip_id}")

# create GeoDataFrame with empty geometry column
gdf = gpd.GeoDataFrame(trip_data_list)
gdf['geometry'] = None

gdf.head()


C:\Users\mikko\AppData\Local\Temp\ipykernel_13568\1130451628.py:28: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf['geometry'] = None


,id,highlighted_photo_id,highlighted_photo_checksum,distance,elevation_gain,elevation_loss,track_id,user_id,visibility,created_at,...,user,gear,tag_names,track_type,terrain,difficulty,points_of_interest,course_points,track_points,geometry
0,84791301,0,None,4895.63,53.6679,59.1766,622f5f15fa348eb767385b53,207709,0,1647271701,...,None,"{'id': 42654, 'name': 'Giant Road'}",[],point_to_point,unknown,unknown,[],[],"[{'T': 21.0, 'c': 0, 'd': 0.0, 'e': 251.6, 'h'...",None
1,146161671,0,None,5079.13,218.7130,226.8620,65c7071f6b34d7c751b01fd9,6812861,0,1707542303,...,None,None,[],out_and_back,climbing,easy,[],[],"[{'x': 27.451418, 'y': 68.342537, 'e': 344.6, ...",None
2,39591949,0,None,6215.91,132.4050,31.4341,5d734d74bca5926df8000000,1498406,0,1567837556,...,None,None,[],point_to_point,climbing,casual,[],[],"[{'d': 0.0, 'e': 240.4, 's': 0.0, 't': 1567836...",None
3,80611342,0,None,1513.53,69.8789,51.3146,61cc35e46b34d7d08d0330e5,3627469,0,1640773092,...,None,None,[],loop,climbing,casual,[],[],"[{'c': 0, 'd': 0.0, 'e': 309.7999999, 'h': 111...",None
4,3790864,0,None,6360.02,150.8540,153.8390,1f9005cd7261730b77010000,138464,0,1416483934,...,None,None,[],out_and_back,climbing,casual,[],[],"[{'d': 0.0, 'e': 174.2, 's': 0.0, 't': 1416401...",None


##Function to create geometries##

In [83]:
from shapely.geometry import Point, LineString

#function to create linestring from track points
def create_line_geometry(track_points):
    if track_points:
        #empty list for coordinates
        coordinates = []
        #iterate through track point coordinates
        for track in track_points:
            #check of track point contains coordinates
            if 'x' in track and 'y' in track:
                #create point from coordinates
                route_point = Point(track['x'], track['y'])
                #add point to point list
                coordinates.append(route_point)
        #create linestring from list of points
        if len(coordinates) >= 2:
            line = LineString(coordinates)
            return line
        else:
            return None
    else:
        return None

#apply the function
gdf['geometry'] = gdf['track_points'].apply(create_line_geometry)

#drop rows with no geometry
gdf = gdf.dropna(subset=['geometry'])

gdf.head()

,id,highlighted_photo_id,highlighted_photo_checksum,distance,elevation_gain,elevation_loss,track_id,user_id,visibility,created_at,...,user,gear,tag_names,track_type,terrain,difficulty,points_of_interest,course_points,track_points,geometry
0,84791301,0,None,4895.63,53.6679,59.1766,622f5f15fa348eb767385b53,207709,0,1647271701,...,None,"{'id': 42654, 'name': 'Giant Road'}",[],point_to_point,unknown,unknown,[],[],"[{'T': 21.0, 'c': 0, 'd': 0.0, 'e': 251.6, 'h'...","LINESTRING (24.29957 67.59952, 24.29957 67.599..."
1,146161671,0,None,5079.13,218.7130,226.8620,65c7071f6b34d7c751b01fd9,6812861,0,1707542303,...,None,None,[],out_and_back,climbing,easy,[],[],"[{'x': 27.451418, 'y': 68.342537, 'e': 344.6, ...","LINESTRING (27.45142 68.34254, 27.45146 68.342..."
2,39591949,0,None,6215.91,132.4050,31.4341,5d734d74bca5926df8000000,1498406,0,1567837556,...,None,None,[],point_to_point,climbing,casual,[],[],"[{'d': 0.0, 'e': 240.4, 's': 0.0, 't': 1567836...","LINESTRING (24.14044 67.62433, 24.14044 67.624..."
3,80611342,0,None,1513.53,69.8789,51.3146,61cc35e46b34d7d08d0330e5,3627469,0,1640773092,...,None,None,[],loop,climbing,casual,[],[],"[{'c': 0, 'd': 0.0, 'e': 309.7999999, 'h': 111...","LINESTRING (27.42754 68.41750, 27.42755 68.417..."
4,3790864,0,None,6360.02,150.8540,153.8390,1f9005cd7261730b77010000,138464,0,1416483934,...,None,None,[],out_and_back,climbing,casual,[],[],"[{'d': 0.0, 'e': 174.2, 's': 0.0, 't': 1416401...","LINESTRING (24.82481 67.81132, 24.82480 67.811..."


In [84]:
#transform timestamp to datetime
import datetime
gdf['created_at'] = gdf['created_at'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [85]:
gdf.head()

,id,highlighted_photo_id,highlighted_photo_checksum,distance,elevation_gain,elevation_loss,track_id,user_id,visibility,created_at,...,user,gear,tag_names,track_type,terrain,difficulty,points_of_interest,course_points,track_points,geometry
0,84791301,0,None,4895.63,53.6679,59.1766,622f5f15fa348eb767385b53,207709,0,2022-03-14 17:28:21,...,None,"{'id': 42654, 'name': 'Giant Road'}",[],point_to_point,unknown,unknown,[],[],"[{'T': 21.0, 'c': 0, 'd': 0.0, 'e': 251.6, 'h'...","LINESTRING (24.29957 67.59952, 24.29957 67.599..."
1,146161671,0,None,5079.13,218.7130,226.8620,65c7071f6b34d7c751b01fd9,6812861,0,2024-02-10 07:18:23,...,None,None,[],out_and_back,climbing,easy,[],[],"[{'x': 27.451418, 'y': 68.342537, 'e': 344.6, ...","LINESTRING (27.45142 68.34254, 27.45146 68.342..."
2,39591949,0,None,6215.91,132.4050,31.4341,5d734d74bca5926df8000000,1498406,0,2019-09-07 09:25:56,...,None,None,[],point_to_point,climbing,casual,[],[],"[{'d': 0.0, 'e': 240.4, 's': 0.0, 't': 1567836...","LINESTRING (24.14044 67.62433, 24.14044 67.624..."
3,80611342,0,None,1513.53,69.8789,51.3146,61cc35e46b34d7d08d0330e5,3627469,0,2021-12-29 12:18:12,...,None,None,[],loop,climbing,casual,[],[],"[{'c': 0, 'd': 0.0, 'e': 309.7999999, 'h': 111...","LINESTRING (27.42754 68.41750, 27.42755 68.417..."
4,3790864,0,None,6360.02,150.8540,153.8390,1f9005cd7261730b77010000,138464,0,2014-11-20 13:45:34,...,None,None,[],out_and_back,climbing,casual,[],[],"[{'d': 0.0, 'e': 174.2, 's': 0.0, 't': 1416401...","LINESTRING (24.82481 67.81132, 24.82480 67.811..."


In [89]:
#sort by date posted
gdf_sorted = gdf.sort_values(by='created_at')

In [93]:
#limit to >2021
gdf_sorted_2021 = gdf_sorted.loc[gdf_sorted['created_at']>='2021']

In [95]:
len(gdf_sorted_2021)

762

In [86]:
#assign crs
gdf.crs = 'EPSG:4326'

